# Working with Open Model Zoo Models
This tutorial shows how to download a model from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo), convert it to OpenVINO™ IR format, show information about the model, and benchmark the model.

#### Table of contents:
- [OpenVINO and Open Model Zoo Tools](#OpenVINO-and-Open-Model-Zoo-Tools)
- [Preparation](#Preparation)
    - [Model Name](#Model-Name)
    - [Imports](#Imports)
    - [Settings and Configuration](#Settings-and-Configuration)
- [Download a Model from Open Model Zoo](#Download-a-Model-from-Open-Model-Zoo)
- [Convert a Model to OpenVINO IR format](#Convert-a-Model-to-OpenVINO-IR-format)
- [Get Model Information](#Get-Model-Information)
- [Run Benchmark Tool](#Run-Benchmark-Tool)
    - [Benchmark with Different Settings](#Benchmark-with-Different-Settings)


## OpenVINO and Open Model Zoo Tools
[back to top ⬆️](#Table-of-contents:)

OpenVINO and Open Model Zoo tools are listed in the table below.

| Tool             | Command             | Description                                             |
|:-----------------|:--------------------|:--------------------------------------------------------|
| Model Downloader | `omz_downloader`      | Download models from Open Model Zoo.                    |
| Model Converter  | `omz_converter`       | Convert Open Model Zoo models to OpenVINO's IR format.  |
| Info Dumper      | `omz_info_dumper`     | Print information about Open Model Zoo models.          |
| Benchmark Tool   | `benchmark_app`       | Benchmark model performance by computing inference time.|

In [1]:
# Install openvino package
%pip install -q "openvino-dev>=2023.1.0"

Note: you may need to restart the kernel to use updated packages.


## Preparation
[back to top ⬆️](#Table-of-contents:)

### Model Name
[back to top ⬆️](#Table-of-contents:)

Set `model_name` to the name of the Open Model Zoo model to use in this notebook. Refer to the list of [public](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/index.md) and [Intel](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/index.md) pre-trained models for a full list of models that can be used. Set `model_name` to the model you want to use.

In [2]:
# model_name = "resnet-50-pytorch"
model_name = "mobilenet-v2-pytorch"

### Imports
[back to top ⬆️](#Table-of-contents:)


In [3]:
import json
from pathlib import Path

import openvino as ov
from IPython.display import Markdown, display

# Fetch `notebook_utils` module
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)
from notebook_utils import DeviceNotFoundAlert, NotebookAlert

### Settings and Configuration
[back to top ⬆️](#Table-of-contents:)

Set the file and directory paths. By default, this notebook downloads models from Open Model Zoo to the `open_model_zoo_models` directory in your `$HOME` directory. On Windows, the $HOME directory is usually `c:\users\username`, on Linux `/home/username`. To change the folder, change `base_model_dir` in the cell below.

The following settings can be changed:

* `base_model_dir`: Models will be downloaded into the `intel` and `public` folders in this directory.
* `omz_cache_dir`: Cache folder for Open Model Zoo. Specifying a cache directory is not required for Model Downloader and Model Converter, but it speeds up subsequent downloads.
* `precision`: If specified, only models with this precision will be downloaded and converted.

In [4]:
base_model_dir = Path("model")
omz_cache_dir = Path("cache")
precision = "FP16"

# Check if an iGPU is available on this system to use with Benchmark App.
core = ov.Core()
gpu_available = "GPU" in core.available_devices

print(
    f"base_model_dir: {base_model_dir}, omz_cache_dir: {omz_cache_dir}, gpu_availble: {gpu_available}"
)

base_model_dir: model, omz_cache_dir: cache, gpu_availble: True


## Download a Model from Open Model Zoo
[back to top ⬆️](#Table-of-contents:)



Specify, display and run the Model Downloader command to download the model.

In [5]:
## Uncomment the next line to show help in omz_downloader which explains the command-line options.

# !omz_downloader --help

In [6]:
download_command = (
    f"omz_downloader --name {model_name} --output_dir {base_model_dir} --cache_dir {omz_cache_dir}"
)
display(Markdown(f"Download command: `{download_command}`"))
display(Markdown(f"Downloading {model_name}..."))
! $download_command

Download command: `omz_downloader --name mobilenet-v2-pytorch --output_dir model --cache_dir cache`

Downloading mobilenet-v2-pytorch...

################|| Downloading mobilenet-v2-pytorch ||################

========== Downloading model/public/mobilenet-v2-pytorch/mobilenet_v2-b0353104.pth
... 100%, 13879 KB, 5839 KB/s, 2 seconds passed



## Convert a Model to OpenVINO IR format
[back to top ⬆️](#Table-of-contents:)

Specify, display and run the Model Converter command to convert the model to OpenVINO IR format. Model conversion may take a while. The output of the Model Converter command will be displayed. When the conversion is successful, the last lines of the output will include: `[ SUCCESS ] Generated IR version 11 model.` For downloaded models that are already in OpenVINO IR format, conversion will be skipped.

In [20]:
## Uncomment the next line to show Help in omz_converter which explains the command-line options.

!omz_converter --help

usage: omz_converter [-h] [-d DIR] [-o DIR] [--name PAT[,PAT...]]
                     [--list FILE.LST] [--all] [--print_all]
                     [--precisions PREC[,PREC...]] [-p PYTHON] [--mo MO.PY]
                     [--add_mo_arg ARG] [--dry_run] [-j JOBS]

options:
  -h, --help            show this help message and exit
  -d DIR, --download_dir DIR
                        root of the directory tree with downloaded model files
  -o DIR, --output_dir DIR
                        root of the directory tree to place converted files
                        into
  --name PAT[,PAT...]   convert only models whose names match at least one of
                        the specified patterns
  --list FILE.LST       convert only models whose names match at least one of
                        the patterns in the specified file
  --all                 convert all available models
  --print_all           print all available models
  --precisions PREC[,PREC...]
                        run only 

In [9]:
convert_command = f"omz_converter --name {model_name} --precisions {precision} --download_dir {base_model_dir} --output_dir {base_model_dir}"
display(Markdown(f"Convert command: `{convert_command}`"))
display(Markdown(f"Converting {model_name}..."))

! $convert_command

Convert command: `omz_converter --name mobilenet-v2-pytorch --precisions FP16 --download_dir model --output_dir model`

Converting mobilenet-v2-pytorch...

========== Converting mobilenet-v2-pytorch to ONNX
Conversion to ONNX command: /home/user/.openvino_env/bin/python3 -- /home/user/.openvino_env/lib/python3.10/site-packages/openvino/model_zoo/internal_scripts/pytorch_to_onnx.py --model-name=mobilenet_v2 --weights=model/public/mobilenet-v2-pytorch/mobilenet_v2-b0353104.pth --import-module=torchvision.models --input-shape=1,3,224,224 --output-file=model/public/mobilenet-v2-pytorch/mobilenet-v2.onnx --input-names=data --output-names=prob

ONNX check passed successfully.

========== Converting mobilenet-v2-pytorch to IR (FP16)
Conversion command: /home/user/.openvino_env/bin/python3 -- /home/user/.openvino_env/bin/mo --framework=onnx --output_dir=model/public/mobilenet-v2-pytorch/FP16 --model_name=mobilenet-v2-pytorch --input=data '--mean_values=data[123.675,116.28,103.53]' '--scale_values=data[58.624,57.12,57.375]' --reverse_input_channels --output=prob --input_model=model/public/mobilenet-v2-pytorch/mobilenet-v2.onnx '--layout=data(NCHW)

## Get Model Information
[back to top ⬆️](#Table-of-contents:)

The Info Dumper prints the following information for Open Model Zoo models:

* Model name
* Description
* Framework that was used to train the model
* License URL
* Precisions supported by the model
* Subdirectory: the location of the downloaded model
* Task type

This information can be shown by running `omz_info_dumper --name model_name` in a terminal. The information can also be parsed and used in scripts. 

In the next cell, run Info Dumper and use `json` to load the information in a dictionary. 

In [10]:
model_info_output = %sx omz_info_dumper --name $model_name
model_info = json.loads(model_info_output.get_nlstr())

if len(model_info) > 1:
    NotebookAlert(
        f"There are multiple IR files for the {model_name} model. The first model in the "
        "omz_info_dumper output will be used for benchmarking. Change "
        "`selected_model_info` in the cell below to select a different model from the list.",
        "warning",
    )

model_info

[{'name': 'mobilenet-v2-pytorch',
  'composite_model_name': None,
  'description': 'MobileNet V2 is image classification model pre-trained on ImageNet dataset. This is a PyTorch* implementation of MobileNetV2 architecture as described in the paper "Inverted Residuals and Linear Bottlenecks: Mobile Networks for Classification, Detection and Segmentation" <https://arxiv.org/abs/1801.04381>.\nThe model input is a blob that consists of a single image of "1, 3, 224, 224" in "RGB" order.\nThe model output is typical object classifier for the 1000 different classifications matching with those in the ImageNet database.',
  'framework': 'pytorch',
  'license_url': 'https://raw.githubusercontent.com/pytorch/vision/master/LICENSE',
  'accuracy_config': '/home/user/.openvino_env/lib/python3.10/site-packages/openvino/model_zoo/models/public/mobilenet-v2-pytorch/accuracy-check.yml',
  'model_config': '/home/user/.openvino_env/lib/python3.10/site-packages/openvino/model_zoo/models/public/mobilenet-v2

Having information of the model in a JSON file enables extraction of the path to the model directory, and building the path to the OpenVINO IR file.

In [11]:
selected_model_info = model_info[0]
model_path = (
    base_model_dir
    / Path(selected_model_info["subdirectory"])
    / Path(f"{precision}/{selected_model_info['name']}.xml")
)
print(model_path, "exists:", model_path.exists())

model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml exists: True


## Run Benchmark Tool
[back to top ⬆️](#Table-of-contents:)

By default, Benchmark Tool runs inference for 60 seconds in asynchronous mode on CPU. It returns inference speed as latency (milliseconds per image) and throughput values (frames per second). 

In [22]:
## Uncomment the next line to show Help in benchmark_app which explains the command-line options.
!benchmark_app --help

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
usage: benchmark_app [-h [HELP]] [-i PATHS_TO_INPUT [PATHS_TO_INPUT ...]] -m
                     PATH_TO_MODEL [-d TARGET_DEVICE]
                     [-hint {throughput,tput,cumulative_throughput,ctput,latency,none}]
                     [-niter NUMBER_ITERATIONS] [-t TIME] [-b BATCH_SIZE]
                     [-shape SHAPE] [-data_shape DATA_SHAPE] [-layout LAYOUT]
                     [-extensions EXTENSIONS] [-c PATH_TO_CLDNN_CONFIG]
                     [-cdir CACHE_DIR] [-lfile [LOAD_FROM_FILE]]
                     [-api {sync,async}] [-nireq NUMBER_INFER_REQUESTS]
                     [-nstreams NUMBER_STREAMS]
                     [-inference_only [INFERENCE_ONLY]]
                     [-infer_precision INFER_PRECISION]
                     [-ip {bool,f16,f32,f64,i8,i16,i32,i64,u8,u16,u32,u64}]
                     [-op {bool,f16,f32,f64,i8,i16,i32,i64,u8,u16,u32,u64}]
                     [-

In [23]:
benchmark_command = f"benchmark_app -m {model_path} -t 15 -d GPU"
display(Markdown(f"Benchmark command: `{benchmark_command}`"))
display(Markdown(f"Benchmarking {model_name} on CPU with async inference for 15 seconds..."))

! $benchmark_command

Benchmark command: `benchmark_app -m model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml -t 15 -d GPU`

Benchmarking mobilenet-v2-pytorch on CPU with async inference for 15 seconds...

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2023.2.0-13089-cfd42bd2cb0-HEAD
[ INFO ] 
[ INFO ] Device info:
[ INFO ] GPU
[ INFO ] Build ................................. 2023.2.0-13089-cfd42bd2cb0-HEAD
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(GPU) performance hint will be set to PerformanceMode.THROUGHPUT.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 7.26 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     data (node: data) : f32 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     prob (node: prob) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO

### Benchmark with Different Settings
[back to top ⬆️](#Table-of-contents:)


The `benchmark_app` tool displays logging information that is not always necessary. A more compact result is achieved when the output is parsed with `json`.

The following cells show some examples of `benchmark_app` with different parameters. Below are some useful parameters:

- `-d` A device to use for inference. For example: CPU, GPU, MULTI. Default: CPU.
- `-t` Time expressed in number of seconds to run inference. Default: 60.
- `-api` Use asynchronous (async) or synchronous (sync) inference. Default: async.
- `-b` Batch size. Default: 1.


Run `! benchmark_app --help` to get an overview of all possible command-line parameters.

In the next cell, define the `benchmark_model()` function that calls `benchmark_app`. This makes it easy to try different combinations. In the cell below that, you display available devices on the system.

> **Note**: In this notebook, `benchmark_app` runs for 15 seconds to give a quick indication of performance. For more accurate performance, it is recommended to run inference for at least one minute by setting the `t` parameter to 60 or higher, and run `benchmark_app` in a terminal/command prompt after closing other applications. Copy the **benchmark command** and paste it in a command prompt where you have activated the `openvino_env` environment. 

In [14]:
def benchmark_model(model_xml, device="CPU", seconds=60, api="async", batch=1):
    core = ov.Core()
    model_path = Path(model_xml)
    if ("GPU" in device) and ("GPU" not in core.available_devices):
        DeviceNotFoundAlert("GPU")
    else:
        benchmark_command = f"benchmark_app -m {model_path} -d {device} -t {seconds} -api {api} -b {batch}"
        display(Markdown(f"**Benchmark {model_path.name} with {device} for {seconds} seconds with {api} inference**"))
        display(Markdown(f"Benchmark command: `{benchmark_command}`"))

        benchmark_output = %sx $benchmark_command
        print("command ended")
        benchmark_result = [line for line in benchmark_output
                            if not (line.startswith(r"[") or line.startswith("      ") or line == "")]
        print("\n".join(benchmark_result))

In [15]:
core = ov.Core()

# Show devices available for OpenVINO Runtime
for device in core.available_devices:
    device_name = core.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

CPU: 13th Gen Intel(R) Core(TM) i5-13400
GPU: Intel(R) Graphics [0x4682] (iGPU)


In [16]:
benchmark_model(model_path, device="CPU", seconds=15, api="async")

**Benchmark mobilenet-v2-pytorch.xml with CPU for 15 seconds with async inference**

Benchmark command: `benchmark_app -m model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml -d CPU -t 15 -api async -b 1`

command ended



In [17]:
benchmark_model(model_path, device="AUTO", seconds=15, api="async")

**Benchmark mobilenet-v2-pytorch.xml with AUTO for 15 seconds with async inference**

Benchmark command: `benchmark_app -m model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml -d AUTO -t 15 -api async -b 1`

command ended



In [18]:
benchmark_model(model_path, device="GPU", seconds=15, api="async")

**Benchmark mobilenet-v2-pytorch.xml with GPU for 15 seconds with async inference**

Benchmark command: `benchmark_app -m model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml -d GPU -t 15 -api async -b 1`

command ended



In [19]:
benchmark_model(model_path, device="MULTI:CPU,GPU", seconds=15, api="async")

**Benchmark mobilenet-v2-pytorch.xml with MULTI:CPU,GPU for 15 seconds with async inference**

Benchmark command: `benchmark_app -m model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml -d MULTI:CPU,GPU -t 15 -api async -b 1`

command ended

